In [1]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_bayesian_regret_fixed_cost_runs(acq, dim, kernel, lengthscale):
    runs = api.runs(path="ziv-scully-group/PandoraBayesOPT", filters={
        "sweep": "36h918l7",
        "config.amplitude": 1,
        "config.dim": dim,
        "config.kernel": kernel,
        "config.lengthscale": lengthscale,  
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        summary_metric_keys = ["global optimum value"]
        summary_history = run.scan_history(keys = summary_metric_keys, page_size=1_000_000_000)
        metrics.update({k: [d[k] for d in summary_history] for k in summary_metric_keys})
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [2]:
acquisition_functions = {
    'UpperConfidenceBound': 'UpperConfidenceBound',
    }
dimensions = [4, 8, 16, 32]
kernels = ["Matern32", "Matern52", "RBF"]
lengthscales = [1.0, 0.1]

In [3]:
grouped_runs = {(a,d,k,l): load_bayesian_regret_fixed_cost_runs(a,d,k,l) for a in acquisition_functions.keys() for d in (dimensions) for k in (kernels) for l in (lengthscales)}

 88%|████████▊ | 14/16 [00:18<00:02,  1.39s/it]wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=19 sec). Create a new API with an integer timeout larger than 19, e.g., `api = wandb.Api(timeout=29)` to increase the graphql timeout.
wandb: Network error (ConnectionError), entering retry loop.
100%|██████████| 16/16 [00:19<00:00,  1.24s/it]


In [4]:
for a in acquisition_functions.keys():
    for d in dimensions:
        for k in kernels:
            for l in lengthscales:
                config_and_metrics_per_seed = grouped_runs[a,d,k,l]
        
                cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T
                best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T
                # Handling potential empty arrays
                if cumulative_cost_per_seed.size == 0 or best_observed_per_seed.size == 0:
                    continue  # Skip this iteration if there's no data
                global_optimum_per_seed = np.array([m['global optimum value'][0] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0])
        
                regret_per_seed = global_optimum_per_seed - best_observed_per_seed
                print(a, d, k, l, regret_per_seed.shape)
        
                regret_25 = np.quantile(regret_per_seed, 0.25, axis=1)
                regret_50 = np.quantile(regret_per_seed, 0.5, axis=1)
                regret_75 = np.quantile(regret_per_seed, 0.75, axis=1)
        
                output = np.stack((cumulative_cost_per_seed.mean(axis=1), regret_25, regret_50, regret_75),axis=-1)
        
                np.savetxt(f"results/fixed_amplitude/BayesianRegret_FixedCost_d{d}_{k}_ls{l}_{acquisition_functions[a]}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

UpperConfidenceBound 4 Matern32 1.0 (41, 16)
UpperConfidenceBound 4 Matern32 0.1 (101, 16)
UpperConfidenceBound 4 Matern52 1.0 (41, 16)
UpperConfidenceBound 4 Matern52 0.1 (101, 16)
UpperConfidenceBound 4 RBF 1.0 (21, 16)
UpperConfidenceBound 4 RBF 0.1 (81, 16)
UpperConfidenceBound 8 Matern32 1.0 (81, 16)
UpperConfidenceBound 8 Matern32 0.1 (201, 16)
UpperConfidenceBound 8 Matern52 1.0 (81, 16)
UpperConfidenceBound 8 Matern52 0.1 (201, 16)
UpperConfidenceBound 8 RBF 1.0 (41, 16)
UpperConfidenceBound 8 RBF 0.1 (161, 16)
UpperConfidenceBound 16 Matern32 1.0 (161, 16)
UpperConfidenceBound 16 Matern32 0.1 (401, 16)
UpperConfidenceBound 16 Matern52 1.0 (161, 16)
UpperConfidenceBound 16 Matern52 0.1 (401, 16)
UpperConfidenceBound 16 RBF 1.0 (81, 16)
UpperConfidenceBound 16 RBF 0.1 (321, 16)
UpperConfidenceBound 32 Matern32 1.0 (321, 16)
UpperConfidenceBound 32 Matern32 0.1 (801, 16)
UpperConfidenceBound 32 Matern52 1.0 (321, 16)
UpperConfidenceBound 32 Matern52 0.1 (801, 16)
UpperConfidenceB